In [7]:
import nltk
import re
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from string import punctuation

#stopwords em portugues
from nltk.corpus import stopwords
stopwords = set(stopwords.words('portuguese') + list(punctuation) + ['“','”'])

#funçao pra limpeza de texto
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)

In [8]:
#funçao pra leitura de dados e criaçao de dataframe
def json_to_dataframe(nome_json, colunas, chunk):
    dados = pd.read_json(nome_json, convert_dates = False, lines = True, chunksize = chunk)    
    
    for df_tweets_retweets in dados:
        yield df_tweets_retweets[colunas]

In [9]:
#funçao pra limpar texto e retirar stopwords
def tokenizar_texto(texto_tweet):
    texto_tokenizado = tweet_tokenizer.tokenize(texto_tweet)

    if texto_tokenizado[0] == "rt" and texto_tokenizado[1] == ":":
        del texto_tokenizado[0]
    
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado if palavra not in stopwords] 
    return palavras_sem_stopwords

In [18]:
#funçao pra criar dicionario com a data e quantidade das palavras chaves
def contagem_pchaves_data(pchaves_lista, gerador):
    
    dict_datas_pchaves = defaultdict(dict)            
    
    for dataframe in gerador:        
        for linha in dataframe.values:            
            texto_tweet = list(tokenizar_texto(linha[0]))

            for palavra in texto_tweet:                
                if palavra in pchaves_lista:
                    data = linha[1]['$date'][:10]
                    if data not in dict_datas_pchaves or palavra not in dict_datas_pchaves[data]:
                        dict_datas_pchaves[data][palavra] = 0
                    dict_datas_pchaves[data][palavra] += 1 
            clear_output()
            
    escrever_arq_csv(dict_datas_pchaves)

In [20]:
#funçao pra escrever os resultados no arquivo csv
def escrever_arq_csv(dict_datas_pchaves):
    with open('cont_hidro_cloro_data.csv', 'w', encoding="utf-8") as f:
        f.write("Data;Palavra;Ocorrencias\n")
    
        for key in dict_datas_pchaves.keys():
            for palavra in dict_datas_pchaves[key].keys():
                f.write("%s;%s;%s\n"%(key, palavra, dict_datas_pchaves[key][palavra]))
            
    print('A contagem de palavras chaves por data foi salva com sucesso no arquivo')

In [21]:
#execuçao do codigo
gerador = json_to_dataframe('5k_origin.json',["text","created_at"], 30)

contagem_pchaves_data(['hidroxicloroquina', 'cloroquina'], gerador)

A contagem de palavras chaves por data foi salva com sucesso no arquivo
